In [3]:
import numpy as np
import pandas as pd
import tensorflow
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
import csv
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score



In [4]:

trainy=pd.read_csv('y_train.csv')
train=pd.read_csv('X_train.csv')
print(train)



       pixel0  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  pixel8  \
0           0       0       0       0       0       0       0       0       0   
1           0       0       0       0       0       0       0       0       0   
2           0       0       0       0       0       0       0       0       0   
3           0       0       0       0       0       0       0       0       0   
4           0       0       0       0       0       0       0       0       0   
...       ...     ...     ...     ...     ...     ...     ...     ...     ...   
37928       0       0       0       0       0       0       0       0       0   
37929       0       0       0       0       0       0       0       0       0   
37930       0       0       0       0       0       0       0       0       0   
37931       3      11       0       0       0       0       0       0       0   
37932       0       0       0       0       0       0       0       0       0   

       pixel9  ...  pixel10

In [5]:
train.insert(1024, "label", list(trainy.label.values-1), True)
np.random.shuffle(train.values)
trainx=train.copy()

del trainx["label"]
sc = StandardScaler()
trainx = sc.fit_transform(trainx)
trainy=train.loc[:,"label"]
print(trainy)



0         8
1        14
2         7
3         4
4         6
         ..
37928    22
37929    23
37930    15
37931    24
37932     7
Name: label, Length: 37933, dtype: int64


In [6]:
trainy=np.asarray(trainy)
ohe = OneHotEncoder(sparse=False)
trainy=trainy.reshape(len(trainy),1)
trainy = ohe.fit_transform(trainy)
print(trainy)
X_train, X_test, Y_train, Y_test = train_test_split(trainx,trainy, test_size=0.01)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [38]:
lay=int(input("How many layers do you want the network to have (min 2)? "))
methodopt=int(input("Which hyperparameter optimization method to use? (1 for seperate, 2 for random) "))
bestact=''
bestunits=[50 for k in range(lay)]
bestacc=0
if methodopt==1:

  act=('relu','sigmoid','tanh')

  for j in act:
    model=keras.models.Sequential()

    model.add(keras.layers.Flatten(input_dim=(1024)))

    for i in range((lay-2)):
        model.add(keras.layers.Dense(units=50,activation=j))

    model.add(keras.layers.Dense(units=29,activation='sigmoid'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy','mse','mae'])

    model.fit(X_train,Y_train,epochs=20,batch_size=200)

    y_pred=model.predict(X_test)
    pred = list()
    for i in range(len(y_pred)):
        pred.append(np.argmax(y_pred[i]))
    test = list()
    for i in range(len(Y_test)):
        test.append(np.argmax(Y_test[i]))
    acc=accuracy_score(test,pred)
    if acc>bestacc:
      bestacc=acc
      bestact=j


  nounits=(100,150,200,250,300)
  for j in nounits:
    model=keras.models.Sequential()

    model.add(keras.layers.Flatten(input_dim=(1024)))

    for i in range((lay-2)):
        model.add(keras.layers.Dense(units=j,activation=bestact))

    model.add(keras.layers.Dense(units=29,activation='sigmoid'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy','mse','mae'])

    model.fit(X_train,Y_train,epochs=20,batch_size=200)

    y_pred=model.predict(X_test)
    pred = list()
    for i in range(len(y_pred)):
        pred.append(np.argmax(y_pred[i]))
    test = list()
    for i in range(len(Y_test)):
        test.append(np.argmax(Y_test[i]))
    acc=accuracy_score(test,pred)
    if acc>bestacc:
      bestacc=acc
      bestunits=[j for k in range(lay)]


else:
  act=['relu','sigmoid','tanh']
  for rnt in range(100):
    tact=act[random.randrange(0,3)]
    nounits=[random.randrange(50,300) for k in range(lay-2)]

    model=keras.models.Sequential()

    model.add(keras.layers.Flatten(input_dim=(1024)))

    for i in nounits:
        model.add(keras.layers.Dense(units=i,activation=tact))

    model.add(keras.layers.Dense(units=29,activation='sigmoid'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy','mse','mae'])

    model.fit(X_train,Y_train,epochs=20,batch_size=200)

    y_pred=model.predict(X_test)
    pred = list()
    for i in range(len(y_pred)):
        pred.append(np.argmax(y_pred[i]))
    test = list()
    for i in range(len(Y_test)):
        test.append(np.argmax(Y_test[i]))
    acc=accuracy_score(test,pred)
    if acc>bestacc:
      bestacc=acc
      bestunits=nounits.copy()
      bestact=tact

model=keras.models.Sequential()

model.add(keras.layers.Flatten(input_dim=(1024)))

#print(bestact)

for i in bestunits:
    model.add(keras.layers.Dense(units=i,activation=bestact))

model.add(keras.layers.Dense(units=29,activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy','mse','mae'])

model.fit(X_train,Y_train,epochs=25,batch_size=200)

y_pred=model.predict(X_test)
pred = list()
for i in range(len(y_pred)):
    pred.append(np.argmax(y_pred[i]))
test = list()
for i in range(len(Y_test)):
    test.append(np.argmax(Y_test[i]))
acc=accuracy_score(test,pred)

print(acc,bestact,bestunits)

How many layers do you want the network to have (min 2)? 4
Which hyperparameter optimization method to use? (1 for seperate, 2 for random) 1
Epoch 1/20
188/188 [==============================] - 2s 4ms/step - loss: 3.1610 - accuracy: 0.1258 - mse: 0.2761 - mae: 0.5035
Epoch 2/20
188/188 [==============================] - 1s 4ms/step - loss: 2.6640 - accuracy: 0.2375 - mse: 0.2883 - mae: 0.4763
Epoch 3/20
188/188 [==============================] - 1s 4ms/step - loss: 2.3774 - accuracy: 0.3018 - mse: 0.2895 - mae: 0.4539
Epoch 4/20
188/188 [==============================] - 1s 5ms/step - loss: 2.1861 - accuracy: 0.3489 - mse: 0.2856 - mae: 0.4337
Epoch 5/20
188/188 [==============================] - 1s 5ms/step - loss: 2.0535 - accuracy: 0.3864 - mse: 0.2808 - mae: 0.4182
Epoch 6/20
188/188 [==============================] - 1s 4ms/step - loss: 1.9583 - accuracy: 0.4108 - mse: 0.2742 - mae: 0.4043
Epoch 7/20
188/188 [==============================] - 1s 4ms/step - loss: 1.8742 - accuracy

In [39]:
testX=pd.read_csv('X_test.csv')
testy=pd.read_csv('y_test.csv')

testX = sc.transform(testX)

testy=np.asarray(testy)
testy=testy.reshape(len(testy),1)
#testy = ohe.transform(testy)

y_pred = model.predict(testX)
#Converting predictions to label
pred = list()
for i in range(len(y_pred)):
    pred.append(np.argmax(y_pred[i]))
#Converting one hot encoded test label to label
test = list()
for i in range(len(testy)):
    test.append(testy[i][0]-1)
print(pred)
print(test)


297/297 [==============================] - 1s 2ms/step
[0, 0, 0, 0, 0, 0, 0, 15, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 22, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 27, 0, 0, 0, 0, 14, 0, 28, 0, 0, 0, 8, 0, 0, 0, 0, 0, 3, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 24, 0, 17, 8, 0, 8, 17, 0, 0, 0, 0, 0, 0, 7, 0, 28, 0, 0, 0, 0, 0, 22, 4, 0, 0, 4, 0, 0, 0, 15, 0, 22, 0, 0, 18, 0, 17, 0, 0, 28, 28, 0, 0, 4, 0, 0, 0, 17, 28, 0, 10, 15, 0, 28, 22, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 21, 0, 0, 0, 28, 9, 0, 0, 0, 0, 0, 0, 8, 17, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 28, 0, 0, 0, 0, 0, 5, 12, 0, 0, 0, 10, 7, 0, 0, 0, 0, 0, 11, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 25, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 25, 0, 0, 0, 0, 0, 0, 7, 0, 0, 3, 10, 0, 6, 0, 8, 0, 0, 22, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 8, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 8, 0, 0, 10, 0, 0, 0, 8, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 22, 0, 22, 0, 0, 0, 0, 7, 0, 16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2

In [40]:
# evaluation :
#confusion matrix
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

#precision
from sklearn.metrics import precision_score

print(f"Precision Score of the classifier is: {precision_score(test, pred,average='macro')}")


# recall
from sklearn.metrics import recall_score

# Calculating the recall score of classifier
print(f"Recall Score of the classifier is: {recall_score(test, pred,average='macro')}")

Precision Score of the classifier is: 0.4761174463653346
Recall Score of the classifier is: 0.4700170549467165
